## 実践タスク: Landsatのベストピクセル抽出
### 課題: Landsat 8/9の画像コレクションから、過去1年間の間に撮影された画像の中から、雲量が最も少ない（ベストピクセル）画像を取得し、その情報を確認しなさい。

1. 課題: Landsat 8/9の画像コレクションから、過去1年間の間に撮影された画像の中から、雲量が最も少ない（ベストピクセル）画像を取得し、その情報を確認しなさい。

|   項目	         | 変更点/値	                                         | 目的                                             |
|----------------|---------------------------------------------------|------------------------------------------|
|データセットID	 | LANDSAT/LC09/C02/T1_L2                    　　　　 | Landsat 9 (Collection 2, Level 2) を使用  |
|対象期間	     | 過去1年間（2024年11月17日 〜 2025年11月17日）　　　　　  | 期間を長くして画像数を増やす                 |
|雲量上限         | 上限は設定しない	                                 | 全ての画像を評価対象とする                   |
|対象地域	     | 琵琶湖の中心 (ee.Geometry.Point([136.17, 35.10] ))	  | 前回と同じ                               | 
|新しい機能	     | .sort() と .first()	                             | メタデータで画像を並べ替え、最も良いものを選択   |

---

In [ ]:
import ee
import datetime

In [ ]:
ee.Initialize(project='earth-change-analysis')

## 1. 地域の定義

In [ ]:
biwako_point = ee.Geometry.Point([136.17, 35.10])

## 2. 検索期間の定義 (過去1年間)

In [ ]:
end_date = '2025-11-17' 
start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

- 検索の終了日を文字列 '2025-11-17' として指定しています。
- これは「2025年11月17日」を意味します。

- **strptime** は「文字列を日付型 (datetime) に変換する」関数です。
- '2025-11-17' を datetime型 に変換します。
→ これで「計算できる日付オブジェクト」になります。

- **timedelta(days=365)** は「365日分の時間差」を表すオブジェクトです。
- - を付けることで「終了日から365日前」を計算します。
→ つまり「開始日 = 終了日 - 365日」。

- **strftime** は「datetime型を文字列に変換する」関数です。
- %Y-%m-%d 形式（例: 2024-11-17）で文字列に戻します。


In [ ]:
print(f"検索期間: {start_date} から {end_date}")

## 3. 画像コレクションのロード

In [ ]:
# Landsat 9 Collection 2, Level 2 のデータセットを使用
l9_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

## 4. フィルタリングと並べ替え

In [ ]:
# a. 日付と範囲でフィルタリング
filtered_collection = l9_collection \
    .filterDate(start_date, end_date) \
    .filterBounds(biwako_point)

In [ ]:
# b. 雲量プロパティ (`CLOUD_COVER`) で昇順に並べ替え
# 昇順 (True): 雲量が少ないもの（ベストなもの）がリストの先頭に来る
sorted_collection = filtered_collection.sort('CLOUD_COVER', True)

In [ ]:
# c. 最も雲量の少ない画像 (`.first()`) を取得
best_image = sorted_collection.first()

## 5. 結果の確認

In [ ]:
# 抽出された画像の総数を取得
count = filtered_collection.size().getInfo()
print(f"最初にフィルタリングされた画像の総数: {count} 枚")

In [ ]:
# 最も良い画像の日付と雲量プロパティを確認
if best_image is not None:
    # 日付の確認
    date_ms = best_image.get('system:time_start').getInfo()
    date_dt = datetime.datetime.fromtimestamp(date_ms / 1000).strftime('%Y-%m-%d')
    
    # 雲量の確認 (Landsat C2 のプロパティは `CLOUD_COVER` です)
    cloud_cover = best_image.get('CLOUD_COVER').getInfo()
    
    print(f"\n ベスト画像の情報:")
    print(f"  - 撮影日: {date_dt}")
    print(f"  - 雲量 (%): {cloud_cover:.2f}%")
else:
    print("\n指定された条件に一致する画像は見つかりませんでした。")

---
## 新しい機能の解説
この課題のポイントは、filterMetadataで閾値を設定しない代わりに、全画像を評価して順位付けを行った点です。

**.sort('CLOUD_COVER', True):**

第1引数は並べ替えの基準となるプロパティ名です。Landsatの雲量プロパティはCLOUD_COVERです（Sentinel-2のCLOUDY_PIXEL_PERCENTAGEとは異なります）。

第2引数 (True) は昇順を意味します。雲量の値が小さいもの（ベストなもの）がコレクションの先頭に来ます。

**.first():**

ソートされたコレクションの先頭にある画像（つまり、最も雲量が少ない画像）をピンポイントで取得します。

この機能を使うことで、手動で雲量の閾値を設定する手間を省き、データセットが提供する中で常に最適な画像を選ぶことができます。
---